In [1]:
import re
import pandas as pd
from io import StringIO

import aidd.sys.config as cfg
from aidd.sys.utils import Logs
from aidd.sys.data_io import read_data, save_data, get_provide_data

In [2]:
class A:
    def __init__(self):
        pass
self = A()

In [3]:
self.pdict = get_provide_data()

[099af5b60638][2024-03-25 19:21:39.042099] 제공받은 데이터 불러오기 시작
[099af5b60638][2024-03-25 19:21:58.320386]   공사비 데이터 셋: 크기(19052, 143), 처리시간(0:00:19.277932)
[099af5b60638][2024-03-25 19:22:15.729633]   전주 데이터 셋: 크기(38533, 63), 처리시간(0:00:17.409167)
[099af5b60638][2024-03-25 19:22:37.805040]   전선 데이터 셋: 크기(40019, 77), 처리시간(0:00:22.075322)
[099af5b60638][2024-03-25 19:22:46.882370]   인입선 데이터 셋: 크기(22632, 57), 처리시간(0:00:09.077218)
[099af5b60638][2024-03-25 19:22:46.882438] 제공받은 데이터 불러오기 종료, 최종 처리시간: 0:01:07.840345


In [4]:
self.is_modeling = True
self.ppdict = {}
self.ppdf = None

In [5]:
key = cfg.DATA_SETs[0]
df = self.pdict[key]
# (전주/전선 수를 제외한) 공사비 데이터 부분에서 학습 대상 레코드 조건
# * 접수종류명(ACC_TYPE_NAME), 계약전력(CONT_CAP), 총공사비(TOTAL_CONS_COST)
modeling_recs = \
    (df.ACC_TYPE_NAME  == cfg.CONSTRAINTs['ACC_TYPE_NAME']) & \
    (df.CONT_CAP        < cfg.CONSTRAINTs['MAX_CONT_CAP']) & \
    (df.TOTAL_CONS_COST < cfg.CONSTRAINTs['MAX_TOTAL_CONS_COST'])
    # (df.CONS_TYPE_CD   == cfg.CONSTRAINTs['CONS_TYPE_CD']) & \
df = df[modeling_recs].reset_index(drop=True)
cons_df = df[cfg.COLs['PP'][key]['SOURCE']]
self.ppdict[key] = cons_df

In [6]:
cons_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15474 entries, 0 to 15473
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CONS_ID          15474 non-null  object        
 1   TOTAL_CONS_COST  15474 non-null  int64         
 2   LAST_MOD_DATE    15474 non-null  datetime64[ns]
 3   OFFICE_NAME      15474 non-null  object        
 4   CONT_CAP         15474 non-null  int64         
 5   ACC_TYPE_NAME    15474 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 725.5+ KB


In [7]:
for key in cfg.DATA_SETs[1:]:
    df = self.pdict[key]
    df = df[df.CONS_ID.isin(cons_df.CONS_ID)]
    self.ppdict[key] = df[cfg.COLs['PP'][key]['SOURCE']]
    print(self.ppdict[key].shape)

(28121, 5)
(30617, 8)
(17222, 5)


In [8]:
if self.is_modeling:
    # 데이터 저장
    for key in cfg.DATA_SETs:
        save_data(self.ppdict[key], f'MERGE,BATCH,{key}')

In [9]:
dt = 'CONS'     # 처리할 데이터 타입(dt)
df = self.ppdict[dt]

# 결측치 처리
df.fillna(0, inplace=True)

In [10]:
# 일자정보 처리
# * '최종변경일시'를 이용해 다양한 일자정보 컬럼 추가
# * 참고로 일자정보가 날자형식이 아니면 날자형식으로 변환
if df.LAST_MOD_DATE.dtype != '<M8[ns]':
    df.LAST_MOD_DATE = pd.to_datetime(df.LAST_MOD_DATE)
df['YEAR'] = df.LAST_MOD_DATE.dt.year
df['MONTH'] = df.LAST_MOD_DATE.dt.month
df['DAY'] = df.LAST_MOD_DATE.dt.day
df['DAYOFWEEK'] = df.LAST_MOD_DATE.dt.dayofweek
df['DAYOFYEAR'] = df.LAST_MOD_DATE.dt.dayofyear
df['YEAR_MONTH'] = df.LAST_MOD_DATE.dt.strftime("%Y%m").astype(int)

In [11]:
if self.is_modeling:
    offc_list = df.OFFICE_NAME.unique().tolist()
    save_data(offc_list, fcode='DUMP,OFFICE_LIST')
else:
    offc_list = read_data('DUMP,OFFICE_LIST')
offc_idxs = []
for oname in df.OFFICE_NAME:
    offc_idxs.append(offc_list.index(oname))
df['OFFICE_NUMBER'] = offc_idxs

In [12]:
df = df[cfg.COLs['PP'][dt]['PP']]
print(df.shape)
self.ppdf = df

(15474, 12)


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15474 entries, 0 to 15473
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CONS_ID          15474 non-null  object        
 1   TOTAL_CONS_COST  15474 non-null  int64         
 2   LAST_MOD_DATE    15474 non-null  datetime64[ns]
 3   OFFICE_NAME      15474 non-null  object        
 4   CONT_CAP         15474 non-null  int64         
 5   YEAR             15474 non-null  int32         
 6   MONTH            15474 non-null  int32         
 7   DAY              15474 non-null  int32         
 8   DAYOFWEEK        15474 non-null  int32         
 9   DAYOFYEAR        15474 non-null  int32         
 10  YEAR_MONTH       15474 non-null  int64         
 11  OFFICE_NUMBER    15474 non-null  int64         
dtypes: datetime64[ns](1), int32(5), int64(4), object(2)
memory usage: 1.1+ MB


In [14]:
ppdf = self.ppdf
# 공사비까지 전처리된 데이터 셋에 설비 갯 수 컬럼 추가(3개)
# 공사비 데이터 셋은 처리하지 않아도 됨
for key in cfg.DATA_SETs[1:]:
    df = self.ppdict[key]
    cons_ids_cnt = df.CONS_ID.value_counts()
    col_name = f'{key}_CNT'
    ppdf = pd.merge(
        ppdf, cons_ids_cnt.rename(col_name),
        left_on='CONS_ID', right_on=cons_ids_cnt.index, how='left'
    )
    # 해당 공사번호가 없는 설비는 NaN처리되기 때문에 이 값을 0으로 변경
    ppdf[col_name] = ppdf[col_name].fillna(0)
print(ppdf.shape)

(15474, 15)


In [15]:
modeling_recs = \
    (ppdf.POLE_CNT >= cfg.CONSTRAINTs['MIN_POLE_CNT']) & \
    (ppdf.POLE_CNT <= cfg.CONSTRAINTs['MAX_POLE_CNT']) & \
    (ppdf.LINE_CNT >= cfg.CONSTRAINTs['MIN_LINE_CNT']) & \
    (ppdf.LINE_CNT <= cfg.CONSTRAINTs['MAX_LINE_CNT'])    
ppdf = ppdf[modeling_recs].reset_index(drop=True)    
print(ppdf.shape)

self.ppdf = ppdf

(14860, 15)


In [16]:
dt = 'POLE'     # 처리할 데이터 타입(dt)
df = self.ppdict[dt]

# 결측치 처리
df.fillna(0, inplace=True)    

In [17]:
# 코드형 컬럼 One-Hot Encoding
prefix = ['POLE_SHAPE', 'POLE_TYPE', 'POLE_SPEC']
cols = [x+'_CD' for x in prefix]
# 숫자형 값 통일(실수형이 아닌 값을 실수형으로 변환)
# (One-Hot Encoding시 동일한 컬럼값을 만들기 위해 실행)
if df.POLE_SPEC_CD.dtype != 'float64':
    df['POLE_SPEC_CD'] = df['POLE_SPEC_CD'].astype(float)
df = pd.get_dummies(df, columns=cols, prefix=prefix)
# True, False값을 1, 0으로 변환
df = df.apply(lambda x: int(x) if isinstance(x, bool) else x)

# 실시간 처리에서 동일 컬럼을 추가하기 위해 학습에서 나온 컬럼 리스트 저장
df_cols = df.columns.tolist()
if self.is_modeling:
    save_data(df_cols, fcode='DUMP,POLE_ONE_HOT_COLS')
else:
    # 학습 당시 컬럼 불러오기
    modeling_cols = read_data(fcode='DUMP,POLE_ONE_HOT_COLS')
    # 실시간 처리에서 만들어 지지 않는 컬럼 추출
    append_cols = [x for x in modeling_cols if x not in df_cols]
    # 0으로 컬럼값 추가
    df.loc[:, append_cols] = 0
print(df.shape)

(28121, 21)


In [18]:
# 공사비별 전주 데이터 합산
unique_cons_ids = df.CONS_ID.unique()
cons_id_pole_sums = []
# 합산대상 컬럼 리스트 추출
sum_cols = [col for col in df.columns if col.startswith('POLE_')]
# 공사번호별 합산(시간이 좀 걸림, 14700건 처리에 약 40초 소요)
for cid in unique_cons_ids:
    cons_id_pole_sums.append(
        [cid]+df[df.CONS_ID==cid][sum_cols].sum().values.tolist())
# 공사번호별로 합산된 전주 정보를 데이터프레임으로 변환
pole_sums_df = pd.DataFrame(
    cons_id_pole_sums, columns=['CONS_ID'] + sum_cols)

# 공사비 데이터와 전주정보 그룹 데이터 병합
ppdf = pd.merge(
    self.ppdf, pole_sums_df,
    left_on='CONS_ID', right_on='CONS_ID', how='left')
print(ppdf.shape)

self.ppdf = ppdf

(14860, 34)


In [19]:
dt = 'LINE'     # 처리할 데이터 타입(dt)
logs = Logs(f'PP_{dt}')
df = self.ppdict[dt]
logs.mid('SOURCE', df.shape)        

# 숫자형 값 통일(실수형이 아닌 값을 실수형으로 변환)
# (One-Hot Encoding시 동일한 컬럼값을 만들기 위해 실행)
if df.LINE_SPEC_CD.dtype != 'float64':
    df['LINE_SPEC_CD'] = df['LINE_SPEC_CD'].astype(float)
if df.NEUTRAL_SPEC_CD.dtype != 'float64':
    df['NEUTRAL_SPEC_CD'] = df['NEUTRAL_SPEC_CD'].astype(float)   
# 중성선규격코드(NEUTRAL_SPEC_CD)에 0.0과 NaN이 존재(NaN=>999.0 변환)
df['NEUTRAL_SPEC_CD'] = df['NEUTRAL_SPEC_CD'].fillna(999.0)
# 중성선종류코드(NEUTRAL_TYPE_CD)의 NaN값을 문자열 'NaN'으로 치환
df.NEUTRAL_TYPE_CD = df.NEUTRAL_TYPE_CD.fillna('NaN')
# 결선방식이 41인 값이 1개만 존재하기 때문에 많이 있는 43으로 치환
df.WIRING_SCHEME = df.WIRING_SCHEME.replace(41, 43)
# 전선 전체길이 추가: = 선로길이(SPAN) * 전선 갯 수(PHASE)
df.loc[:, 'LINE_LENGTH'] = df.SPAN * df.LINE_PHASE_CD

[e1cd3798799c][2024-03-25 19:24:13.409226] 전선 데이터 전처리 시작
[e1cd3798799c][2024-03-25 19:24:13.409888]   전처리 전 전선 데이터 셋 크기: (30617, 8)


In [20]:
# 결측치 처리
df.fillna(0, inplace=True)

# 코드형 컬럼 One-Hot Encoding
# WIRING_SCHEME은 마지막에 '_CD'가 붙지 않음
prefix = ['WIRING_SCHEME', 'LINE_TYPE', 'LINE_SPEC', 'LINE_PHASE',
        'NEUTRAL_TYPE', 'NEUTRAL_SPEC']
columns = [x+'_CD' for x in prefix if x != 'WIRING_SCHEME']
columns += ['WIRING_SCHEME']
df = pd.get_dummies(df, columns=columns, prefix=prefix)
# True, False를 1, 0으로 변환
df = df.apply(lambda x: int(x) if isinstance(x, bool) else x)
# 실시간 처리에서 동일 컬럼을 추가하기 위해 학습에서 나올 컬럼리스트 저장
df_cols = df.columns.tolist()
if self.is_modeling:
    save_data(df_cols, fcode='DUMP,LINE_ONE_HOT_COLS')
else:
    modeling_cols = read_data(fcode='DUMP,LINE_ONE_HOT_COLS')
    append_cols = [col for col in modeling_cols if col not in df_cols]
    df.loc[:, append_cols] = 0
logs.mid('ONE_HOT', df.shape)

[e1cd3798799c][2024-03-25 19:24:39.393916]   전선 데이터 ONE HOT ENCODING 후 데이터 셋 크기: (30617, 45)


In [21]:
# 공사비별 전선 데이터 합산
unique_cons_ids = df.CONS_ID.unique()
cons_id_line_sums = []
sum_cols = ['SPAN'] + df.columns.tolist()[5:]
for cid in unique_cons_ids:
    cons_id_line_sums.append(
        [cid]+df[df.CONS_ID==cid][sum_cols].sum().values.tolist())
# 공사번호별로 합산된 전주 정보를 데이터프레임으로 변환
line_sums_df = pd.DataFrame(
    cons_id_line_sums, columns=['CONS_ID']+sum_cols)

# 공사비 데이터와 전주 그룹 데이터 병합
ppdf = pd.merge(
    self.ppdf, line_sums_df,
    left_on='CONS_ID', right_on='CONS_ID', how='left')
logs.mid('RESULT', ppdf.shape)

self.ppdf = ppdf

[e1cd3798799c][2024-03-25 19:25:45.841059]   전선 데이터 전처리 후 모델링 데이터 셋 크기: (14860, 75)


In [22]:
logs.stop()

[e1cd3798799c][2024-03-25 19:25:45.846589] 전선 데이터 전처리 종료, 최종 처리시간: 0:01:32.437361


In [23]:
dt = 'SL'     # 처리할 데이터 타입(dt)
logs = Logs(f'PP_{dt}')
df = self.ppdict[dt]
logs.mid('SOURCE', df.shape)    

# 숫자형 값 통일(실수형이 아닌 값을 실수형으로 변환)
# (One-Hot Encoding시 동일한 컬럼값을 만들기 위해 실행)
if df.SL_SPEC_CD.dtype != 'float64':
    df['SL_SPEC_CD'] = df['SL_SPEC_CD'].astype(float)
# 결측치 처리
df.fillna(0, inplace=True)

# 코드형 컬럼 One-Hot Encoding
prefix = ['SL_TYPE', 'SL_SPEC']
columns = [col+'_CD' for col in prefix]
df = pd.get_dummies(df, columns=columns, prefix=prefix)
df = df.apply(lambda x: int(x) if isinstance(x, bool) else x)
# 실시간 처리에서 동일 컬럼을 추가하기 위해 학습에서 나올 컬럼리스트 저장
df_cols = df.columns.tolist()
if self.is_modeling:
    save_data(df_cols, fcode='DUMP,SL_ONE_HOT_COLS')
else:
    modeling_cols = read_data('DUMP,SL_ONE_HOT_COLS')
    append_cols = [col for col in modeling_cols if col not in df_cols]
    df.loc[:, append_cols] = 0
logs.mid('ONE_HOT', df.shape)

[4a8f5736555b][2024-03-25 19:26:00.215885] 인입선 데이터 전처리 시작
[4a8f5736555b][2024-03-25 19:26:00.216305]   전처리 전 인입선 데이터 셋 크기: (17222, 5)
[4a8f5736555b][2024-03-25 19:26:00.229186]   인입선 데이터 ONE HOT ENCODING 후 데이터 셋 크기: (17222, 29)


In [25]:
# 공사비별 인입선 데이터 합산
unique_cons_ids = df.CONS_ID.unique()
cons_id_sl_sums = []
sum_cols = df.columns.tolist()[2:]
for cid in unique_cons_ids:
    _df = df[df.CONS_ID==cid]
    sl_sums = _df[sum_cols].sum().values.tolist()
    # sl_comp_id_cnt = _df.COMP_ID.nunique()
    cons_id_sl_sums.append(
        [cid, _df.shape[0]] + sl_sums)
        # [cid, sl_comp_id_cnt, _df.shape[0]] + sl_sums)
# 데이터프레임 만들기
sl_sums_df = pd.DataFrame(
    cons_id_sl_sums, 
    columns=['CONS_ID', 'REAL_SL_CNT', 'SL_SPAN_SUM'] \
        + sum_cols[1:]
)

# 공사비 데이터와 인입선 그룹 데이터 병합
ppdf = pd.merge(
    self.ppdf, sl_sums_df,
    left_on='CONS_ID', right_on='CONS_ID', how='left'
)
logs.mid('RESULT', ppdf.shape)

self.ppdf = ppdf
logs.stop()

[4a8f5736555b][2024-03-25 19:30:46.375976]   인입선 데이터 전처리 후 모델링 데이터 셋 크기: (14860, 103)
[4a8f5736555b][2024-03-25 19:30:46.376238] 인입선 데이터 전처리 종료, 최종 처리시간: 0:04:46.160362


In [26]:
# 최종 완료시점에서 NaN값을 0으로 처리
# 온라인 작업 시 인입선이 없거나 전주가 없는 작업 등에서 NaN가 올 수 있음
self.ppdf.fillna(0, inplace=True)
# 모델링 시점과 서비스 시점의 데이터프레임 컬럼 순서를 동일하게 하기 위해
# 모델링 시점의 컬럼 순서를 저장해 서비스 시점에서 컬럼 순서를 재배치
# One-Hot Encoding시점에 데이터 컬럼의 순서가 변경될 수 있음.
if self.is_modeling:
    last_pp_cols = self.ppdf.columns
    save_data(last_pp_cols, fcode='DUMP,LAST_PP_COLS')
else:
    last_pp_cols = read_data('DUMP,LAST_PP_COLS')
    self.ppdf = self.ppdf.reindex(columns=last_pp_cols)

In [27]:
self.ppdf.shape

(14860, 103)

In [28]:
self.ppdf.columns

Index(['CONS_ID', 'TOTAL_CONS_COST', 'LAST_MOD_DATE', 'OFFICE_NAME',
       'CONT_CAP', 'YEAR', 'MONTH', 'DAY', 'DAYOFWEEK', 'DAYOFYEAR',
       ...
       'SL_SPEC_16.0', 'SL_SPEC_22.0', 'SL_SPEC_25.0', 'SL_SPEC_35.0',
       'SL_SPEC_38.0', 'SL_SPEC_60.0', 'SL_SPEC_70.0', 'SL_SPEC_100.0',
       'SL_SPEC_120.0', 'SL_SPEC_240.0'],
      dtype='object', length=103)

In [29]:
last_pp_cols

Index(['CONS_ID', 'TOTAL_CONS_COST', 'LAST_MOD_DATE', 'OFFICE_NAME',
       'CONT_CAP', 'YEAR', 'MONTH', 'DAY', 'DAYOFWEEK', 'DAYOFYEAR',
       ...
       'SL_SPEC_16.0', 'SL_SPEC_22.0', 'SL_SPEC_25.0', 'SL_SPEC_35.0',
       'SL_SPEC_38.0', 'SL_SPEC_60.0', 'SL_SPEC_70.0', 'SL_SPEC_100.0',
       'SL_SPEC_120.0', 'SL_SPEC_240.0'],
      dtype='object', length=103)